<a href="https://www.bigdatauniversity.com"><img src="https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width="400" align="center"></a>

<h1 align="center"><font size="5">COLLABORATIVE FILTERING</font></h1>

Recommendation systems are a collection of algorithms used to recommend items to users based on information taken from the user. These systems have become ubiquitous can be commonly seen in online stores, movies databases and job finders. In this notebook, we will explore recommendation systems based on Collaborative Filtering and implement simple version of one using Python and the Pandas library.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [25]:
movie = pd.read_csv('moviedataset/movies.csv')
rating = pd.read_csv('moviedataset/ratings.csv')

In [128]:
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [32]:
rating.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


## Data Cleaning and Reorganization

In [129]:
## Put the year information to a new column
movie = movie.drop(['genres'],axis=1)
movie['Year'] = movie.title.str[-5:-1]
movie['title'] = movie.title.str[:-6]
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movie['title'] = movie['title'].apply(lambda x: x.strip())
movie.head()

,movieId,title,Year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


In [33]:
rating = rating.drop(['timestamp'],axis=1)
rating.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


<hr>

<a id="ref3"></a>
# Collaborative Filtering

The first technique we're going to take a look at is called __Collaborative Filtering__, which is also known as __User-User Filtering__. As hinted by its alternate name, this technique uses other users to recommend items to the input user. It attempts to find users that have similar preferences and opinions as the input and then recommends items that they have liked to the input. There are several methods of finding similar users (Even some making use of Machine Learning), and the one we will be using here is going to be based on the __Pearson Correlation Function__.

The process for creating a User Based recommendation system is as follows:
- Select a user with the movies the user has watched
- Based on his rating to movies, find the top X neighbours 
- Get the watched movie record of the user for each neighbour.
- Calculate a similarity score using some formula
- Recommend the items with the highest score

In [126]:
## Define target user
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,rating,title
0,5.0,"Breakfast Club, The"
1,3.5,Toy Story
2,2.0,Jumanji
3,5.0,Pulp Fiction
4,4.5,Akira


In [138]:
## Add movie ID to the inputMovies
# 1. Select target movie from all movie list
TargetList = inputMovies['title'].tolist()
inputId = movie[movie['title'].isin(TargetList)]
inputId

# 2. Merge with traget dataset
inputMovies = inputId.merge(inputMovies, how = 'inner', on = 'title')
inputMovies = inputMovies.drop('Year', axis=1)
inputMovies.head()

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


### The users who has seen the same movies
Now with the movie ID's in our input, we can now get the subset of users that have watched and reviewed the movies in our input.

In [147]:
#Filtering out users that have watched movies that the input has watched and storing it
TargetMovie = inputMovies.movieId.tolist()
userSubset = rating[rating['movieId'].isin(TargetMovie)]
userSubset.head()

,userId,movieId,rating
19,4,296,4.0
441,12,1968,3.0
479,13,2,2.0
531,13,1274,5.0
681,14,296,2.0


In [150]:
# Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
userSubsetGroup = userSubset.groupby(['userId'])

# lets look at one of the users, e.g. the one with userID=1130
userSubsetGroup.get_group(1130)

,userId,movieId,rating
104167,1130,1,0.5
104168,1130,2,4.0
104214,1130,296,4.0
104363,1130,1274,4.5
104443,1130,1968,4.5


In [156]:
#Sorting it so users with movie most in common with the input will have priority
userSubsetGroup = sorted(userSubsetGroup, key = lambda x: len(x[1]),reverse=True) # https://docs.python.org/3/howto/sorting.html
                                          # key is a function that will be called to transform the collection's items before they are compared. 
userSubsetGroup[0:3]

[(75,       userId  movieId  rating
  7507      75        1     5.0
  7508      75        2     3.5
  7540      75      296     5.0
  7633      75     1274     4.5
  7673      75     1968     5.0), (106,       userId  movieId  rating
  9083     106        1     2.5
  9084     106        2     3.0
  9115     106      296     3.5
  9198     106     1274     3.0
  9238     106     1968     3.5), (686,        userId  movieId  rating
  61336     686        1     4.0
  61337     686        2     3.0
  61377     686      296     4.0
  61478     686     1274     4.0
  61569     686     1968     5.0)]

## Similarity of users to input user
Next, we are going to compare all users (not really all !!!) to our specified user and find the one that is most similar.  
we're going to find out how similar each user is to the input through the __Pearson Correlation Coefficient__. It is used to measure the strength of a linear association between two variables. The formula for finding this coefficient between sets X and Y with N values can be seen in the image below. 

Why Pearson Correlation?

Pearson correlation is invariant to scaling, i.e. multiplying all elements by a nonzero constant or adding any constant to all elements. For example, if you have two vectors X and Y,then, pearson(X, Y) == pearson(X, 2 * Y + 3). This is a pretty important property in recommendation systems because for example two users might rate two series of items totally different in terms of absolute rates, but they would be similar users (i.e. with similar ideas) with similar rates in various scales .

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/bd1ccc2979b0fd1c1aec96e386f686ae874f9ec0 "Pearson Correlation")

The values given by the formula vary from r = -1 to r = 1, where 1 forms a direct correlation between the two entities (it means a perfect positive correlation) and -1 forms a perfect negative correlation. 

In our case, a 1 means that the two users have similar tastes while a -1 means the opposite.

In [157]:
userSubsetGroup = userSubsetGroup[0:100]

In [213]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

# Step 1. Access each element of tuple in the list
for ID, group in userSubsetGroup:
    group = group.sort_values(by = 'movieId')
    tempGroupList = group['rating'].tolist()
    
    # Step 2: find the overlapped movie with the target user
    inputMovies = inputMovies.sort_values(by = 'movieId')
    OverlapMovie = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    tempRatingList = OverlapMovie['rating'].tolist()
    
    # Step 3: Calculate the Pearson correlation coefficient
    coeff = np.corrcoef(tempGroupList,tempRatingList)[1,0]
    
    # Step 4: Put the result into dictionary
    pearsonCorrelationDict[ID] = coeff

In [214]:
pearsonCorrelationDict.items()

dict_items([(75, 0.8272781516947569), (106, 0.5860090386731193), (686, 0.8320502943378437), (815, 0.5765566601970551), (1040, 0.9434563530497266), (1130, 0.28915746598312014), (1502, 0.8770580193070293), (1599, 0.43852900965351466), (1625, 0.7161148740394329), (1950, 0.17902871850985821), (2065, 0.43852900965351466), (2128, 0.5860090386731193), (2432, 0.1386750490563073), (2791, 0.8770580193070293), (2839, 0.8204126541423671), (2948, -0.11720180773462387), (3025, 0.45124262819714017), (3040, 0.895143592549291), (3186, 0.6784622064861936), (3271, 0.2698959481797065), (3429, 0.0), (3734, -0.1504142093990467), (4099, 0.05860090386731193), (4208, 0.29417420270727607), (4282, -0.43852900965351466), (4292, 0.6564386345361466), (4415, -0.11183835382312352), (4586, -0.9024852563942803), (4725, -0.08006407690254358), (4818, 0.4885967564883422), (5104, 0.7674257668936506), (5165, -0.43852900965351466), (5547, 0.17200522903844537), (6082, -0.047287799241095906), (6207, 0.9615384615384617), (6366,

In [215]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

,similarityIndex,userId
0,0.827278,75
1,0.586009,106
2,0.832050,686
3,0.576557,815
4,0.943456,1040


## Rating of selected users to all movies
We're going to do this by taking the weighted average of the ratings of the movies using the Pearson Correlation as the weight. But to do this, we first need to get the movies watched by the users in our __pearsonDF__ from the ratings dataframe and then store their correlation in a new column called _similarityIndex". This is achieved below by merging of these two tables.

In [216]:
# Choose top 50 similar users
topUsers = pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

,similarityIndex,userId
64,0.961678,12325
34,0.961538,6207
55,0.961538,10707
67,0.960769,13053
4,0.943456,1040


In [218]:
topUsersRating = topUsers.merge(rating, on = 'userId')
topUsersRating.head()

,similarityIndex,userId,movieId,rating
0,0.961678,12325,1,3.5
1,0.961678,12325,2,1.5
2,0.961678,12325,3,3.0
3,0.961678,12325,5,0.5
4,0.961678,12325,6,2.5


In [222]:
# Multiplies the similarity by the user's ratings
topUsersRating['WeightRating'] = topUsersRating.similarityIndex * topUsersRating.rating
topUsersRating.head()

,similarityIndex,userId,movieId,rating,WeightRating
0,0.961678,12325,1,3.5,3.365874
1,0.961678,12325,2,1.5,1.442517
2,0.961678,12325,3,3.0,2.885035
3,0.961678,12325,5,0.5,0.480839
4,0.961678,12325,6,2.5,2.404196


In [230]:
test = topUsersRating.groupby('movieId')
test.get_group(1).head()

,similarityIndex,userId,movieId,rating,WeightRating
0,0.961678,12325,1,3.5,3.365874
657,0.961538,6207,1,3.5,3.365385
1184,0.961538,10707,1,3.0,2.884615
1916,0.960769,13053,1,4.0,3.843076
3347,0.943456,1040,1,3.0,2.830369


In [227]:
# Applies a sum to the topUsers after grouping it up by movieId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','WeightRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightRating
movieId,,
1,38.376281,140.800834
2,38.376281,96.656745
3,10.253981,27.254477
4,0.929294,2.787882
5,11.723262,27.151751


In [232]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,3.668955,1
2,2.518658,2
3,2.657941,3
4,3.000000,4
5,2.316058,5


In [236]:
top10List = recommendation_df.head(10).movieId.tolist()
movie[movie['movieId'].isin(top10List)] # Equal to: movie.loc[movie['movieId'].isin(top10List)]

,movieId,title,Year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995
5,6,Heat,1995
6,7,Sabrina,1995
7,8,Tom and Huck,1995
8,9,Sudden Death,1995
9,10,GoldenEye,1995
